# Importing Libraries

In [1]:
import os
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Setup the LLM

In [2]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Initialize the text splitter

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=100, add_start_index=True
)

# Load and Split the constitution pdf

In [4]:
file_path = (
    "../constitution.pdf"
)
loader = PyPDFLoader(file_path)
constitution_pages = loader.load_and_split(text_splitter)
len(constitution_pages)

773

# Store in the VectorDB

In [5]:
vectorstore = Chroma.from_documents(documents=constitution_pages, embedding=OpenAIEmbeddings())

In [6]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# System Prompt for the Chain

In [17]:
system_prompt = (
    """ 
    You are an assistant specialized in answering questions solely from the constitution of nepal. 
    Use only the given context to form your responses. 
    If the information is not provided or unclear, state explicitly that you don't know. 
    Keep your answers brief.
    {context}
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Define the RAG Chain

In [18]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Output

In [24]:
response = rag_chain.invoke({"input": "How many articles are there in the constitution"})
print(response["answer"])

There are 45 articles in the Constitution of Nepal.
